# **TP LLM : Large Language Models**

## **Introduction**

Les **LLMs (Large Language Models)** ont révolutionné le domaine de l'Intelligence Artificielle en permettant de réaliser des tâches jusqu'ici considérées complexes : chatbots, traduction automatique, résumé de textes, génération de code...

Démocratisés par **OpenAI** en novembre 2022 avec **ChatGPT**, les LLMs sont rapidement devenus des enjeux stratégiques et économiques pour les entreprises. Aujourd'hui, rares sont les domaines qui ont échappé à leur influence.

IA générative, ChatGPT, LLM... le lexique associé à ces technologies est dense et opaque. Le tableau suivant vise à clarifier ces concepts en les comparant aux composants d'un véhicule, afin de mieux comprendre leur rôle et leur fonctionnement au sein d'une application d'IA générative.

**Composants d'une application d'IA générative :**

| Composant | Analogie véhicule | Rôle | Exemples |
|--|--|--|--|
| LLM | Moteur | Le cœur du système, c’est ce qui fait tourner l'application | GPT-4o / Mistral Large / Claude 3.7 Sonnet / DeepSeek R1 |
| Interface utilisateur (UI) | Carrosserie | Ce que l’utilisateur voit et utilise pour interagir avec le LLM. | ChatGPT / Le Chat / Perplexity |
| Outils | Options | Permettent d’enrichir les capacités de base | Recherche web / Exécution de code / Analyse de documents |

L'objectif de ce TP est d'explorer et de comprendre le fonctionnement des moteurs d'applications d'IA générative, les **LLMs**, afin de mieux appréhender leurs enjeux et leurs limites.

**Plan du TP :**
1. Choix et chargement du LLM
2. Exploration du fonctionnement du LLM
3. Comment transformer un LLM en un assistant ?




## **0. Installations, imports et définition de fonctions**

### Imports et paramètres

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer, logging



# Set device and corresponding data type
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float16 if DEVICE=="cuda" else torch.float32

# Disable torch gradient tracking
torch.set_grad_enabled(False)

# Hide warnings
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
logging.set_verbosity_error()

### Clés d'API

In [ ]:
# Set Hugging Face token
os.environ["HF_TOKEN"] = "" # ask for your token !

### Définition des fonctions

In [ ]:
def plot_token_embeddings(token_ids: list[int]) -> None:
    """
    Plots 2D representations of token embeddings for a given list of token IDs.

    Args:
        model_name (str): Name of the Hugging Face model to load.
        token_ids (list): List of token IDs for which embeddings are extracted.
    """
    # Extract embeddings
    with torch.no_grad():
        embeddings = model.get_input_embeddings()(torch.tensor(token_ids).to(DEVICE))

    # Reduce dimensions to 2D using PCA
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embeddings.cpu().detach().float().numpy())

    # Plot the embeddings with vectors
    plt.figure()
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)  # Add horizontal line at y=0
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)  # Add vertical line at x=0

    for i, token_id in enumerate(token_ids):
        x, y = embeddings_2d[i]
        plt.quiver(0, 0, x, y,
                  angles='xy', scale_units='xy', scale=1,
                  color='blue', alpha=0.5)
        plt.scatter(x, y, color='red', zorder=3)
        plt.text(x, y, tokenizer.decode([token_id]),
                fontsize=12, ha='left', va='bottom')

    plt.title(f"2D Token Embeddings for {model_name}")
    plt.xlabel("PCA Dimension 1")
    plt.ylabel("PCA Dimension 2")
    plt.grid()
    plt.show()

## **1. Choix et chargement du LLM**



En pratique, un LLM se compose de deux fichiers :
- Un fichier de code permettant d'exécuter le modèle, composés de quelques centaines de lignes (~1 Mo)
- Un fichier contenant les paramètres (poids) du modèle. Les plus petits modèles sont composés de quelques milliards de paramètres (1-10 Go) et les plus grands peuvent en posséder plusieurs centaines de milliards (+1 To)

Voici, pour quelques modèles populaires, le nombre de paramètres ainsi que la taille du fichier associé :

| **LLM**               | **Nombre de paramètres** | **Taille du fichier**  |
|--------------------------|--------------------------|------------------------|
| Mistral-7B               | 7 milliards              | 14 Go                  |
| Meta-Llama-3-70B         | 70 milliards             | 140 Go                 |
| GPT-3.5 (ChatGPT 2022)   | 175 milliards            | 350 Go                 |
| DeepSeek-R1              | 685 milliards            | 1370 Go                |

Certaines entreprises publient leurs LLMs en open-source (Meta, Mistral, DeepSeek...), ce qui permet de les télécharger et de les exécuter sur un PC pour les plus petits ou un serveur pour les plus gros. Les autres (OpenAI, Google, Claude...) ne donnent pas accès à leurs modèles rendant toute utilisation *on-premise* impossible.

### **1.1 Découverte d'Hugging Face 🤗**

[Hugging Face 🤗](https://huggingface.co/) est une plate-forme permettant le partage de modèles, de code et de données dans le domaine du **NLP (Natural Language Processing)**, et plus marginalement dans d'autres domaines de l'intelligence artificielle (vision par ordinateur, apprentissage par renforcement). Le site permet à n'importe qui (du simple utilisateur aux grandes entreprises comme **Meta** ou **Mistral**) de partager son travail pour qu'il soit utilisable par la communauté.

Pour explorer les modèles disponibles, rendez-vous dans l'onglet [models](https://huggingface.co/models).

Pour ce TP, nous allons utiliser un "petit" LLM : [Llama-3.2-1B](https://huggingface.co/meta-llama/Llama-3.2-1B), développé par **Meta**.

Il est composé de 1,24 milliards de paramètres et pèse 2,47 Go.



### **1.2 Chargement du modèle et de son tokenizer**

Nous utilisons la librairie python [transformers](https://github.com/huggingface/transformers) développée par Hugging Face qui permet de charger et d'utiliser facilement les modèles disponibles sur la plateforme.

In [ ]:
model_name = "meta-llama/Llama-3.2-1B"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=DTYPE,
    device_map=DEVICE,
    trust_remote_code=True,
)

In [ ]:
# Put model into inference mode
model.eval()

## **2. Exploration du fonctionnement du LLM**

Les LLMs font partie d'une catégorie d'algorithmes nommés **modèles de langage** et dont l'objectif fondamental et de prédire le mot suivant à partir d'une séquence de mots.

Dans cette section, nous allons explorer comment un LLM fonctionne pour prédire le mot suivant. Dans la section suivante, nous expliquerons comment, à partir d'un modèle qui prédit le mot suivant, il est possible de simuler un assistant type ChatGPT.

Le fonctionnement du LLM peut être divisé en 3 étapes :
- tokenizer : découpe le texte brut en une séquence de token lisible par le LLM
- embedding : transforme chaque token en un vecteur numérique qui capture la sémantique du token
- réseau de neurones : génère le token suivant



### **2.1 Tokenizer**

Le tokenizer est un algorithme séparé du LLM. Chaque LLM possède son propose tokenizer. Son objectif est de transformer le texte brut en une série de tokens que le LLM peut comprendre et traiter. Le tokenizer possède un nombre fini de tokens que l'on appelle le **vocabulaire** du LLM.

In [ ]:
# Show LLM vocabulary size
print(f"Vocabulary size: {tokenizer.vocab_size}")

In [ ]:
# Define a prompt
prompt = "RTE est le gestionnaire du réseau de transport d'électricité."

# Try tokenizer functions
tokenized = tokenizer.tokenize(prompt)
encoded = tokenizer.encode(prompt, add_special_tokens=False)

# Check differences between each function
print(f"Tokenized : {[tokenizer.decode(token) for token in encoded]}")
print(f"Encoded : {encoded}")

---

**Question**

Rendez-vous sur [tiktokenizer/Meta-Llama-3-8B](https://tiktokenizer.vercel.app/?model=meta-llama%2FMeta-Llama-3-8B) pour essayer le tokenizer de manière plus interactive.

Essayez également le tokenizer d'un LLM à l'état de l'art, par exemple celui du modèle `GPT-4o` d'OpenAI :  [tiktokenizer/o200k_base](https://tiktokenizer.vercel.app/?model=o200k_base).

Que remarquez-vous ? Quelles conséquences ?

---

### **2.2 Embedding**

Les ordinateurs ne pouvant effectuer des opérations que sur des nombres, il est nécessaire de convertir chaque token en chiffres. Dans les LLMs, cette conversion se fait à l'aide des **embeddings**, des vecteurs numériques qui représentent la sémantique du token : deux tokens qui ont un sens proche sont représentés par des vecteurs numériques similiares.

La première couche du LLM est une simple table de correspondance qui associe à chaque token un embedding.

In [ ]:
# Print LLM embedding layer shape
model.get_input_embeddings()

In [ ]:
# Define a list of waords
word_list = "cat dog green blue house room"

# Plot tokens embeddings in a reduce 2D space
prompt_tokens_ids = list(set(tokenizer.encode(word_list, add_special_tokens=False)))
plot_token_embeddings(prompt_tokens_ids)

---

**Question**

Ajoutez des mots à la liste et observez le comportement.

Quelles limites anticipez-vous avec cette manière de représenter les mots ? Quel va être le rôle du réseau de neurones qui vient après ?

---

### **2.3 Réseau de neurones**

L'essentiel des paramètres du LLM sont contenus dans le réseau de neurones.

Ce réseau de neurone peut-être vu comme un simple classifieur. A partir d'une liste de tokens en entrée, il retourne en sortie la probabilité pour chaque token de son vocabulaire d'être le suivant.



In [ ]:
# Define a prompt and temperature
prompt = "The capital of Australia is"

# Tokenize prompt
inputs = tokenizer.encode(prompt, return_tensors="pt").to(DEVICE)

# Make a forward pass
output = model.forward(input_ids=inputs, use_cache=False)['logits'].squeeze()[-1]

# Show results shape
print(f"Output: {output}")
print(f"Output length: {len(output)}")

In [ ]:
# Define temperature
temperature = 0.5

# Transform model output to probability distribution using softmax
output_softmax = torch.nn.functional.softmax(output/temperature, dim=0)

# Filter top_k tokens
top_k = 10
top_k_indices = torch.topk(output_softmax, top_k).indices
top_k_probabilities = output_softmax[top_k_indices].cpu().detach().float().numpy()
top_k_tokens = [tokenizer.decode([token_id]) for token_id in top_k_indices]

# Barplot
px.bar(x=top_k_tokens, y=top_k_probabilities, labels={"x": "token", "y": "probability"}, title=prompt+"...")

---

**Question**

Un paramètre important est le paramètre `temperature` lors de l'opération softmax.

Modifiez ce paramètre entre 0 et 1 et observez la distribution de sortie.

Quelle influence ce paramètre aura-t-il sur le texte généré par le LLM ?

---

### **2.4 Pipeline de génération**

Pour générer du texte, il suffit de répéter le processus suivant :
- faire une prédiction avec le modèle
- échantillonner la sortie pour sélectionner un token
- ajouter ce token à la séquence de tokens d'entrée

In [ ]:
# Streamer to print token on the fly
streamer = TextStreamer(tokenizer, skip_special_tokens=True)

# Generation pipeline to chain model predictions
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, streamer=streamer)

In [ ]:
# Define prompt
prompt = "The capital of Australia is"

# Generate text continuation
prediction = generator(
    prompt,
    do_sample=True,
    temperature=0.5,
    max_new_tokens=30
)

---

**Question**

Modifiez le paramètre température et observez le texte généré.

Dans quels cas utiliser une température élevée ? Dans quels cas utiliser une température faible ?

---

### **2.5 Limites de taille mémoire**

La taille du prompt impact directement la quantité de mémoire nécessaire pour générer des tokens.

*La cellule suivante ne fonctionne que si le LLM est chargé sur GPU.*

In [ ]:
def measure_memory(token_count: int) -> int:
    """
    Returns total GPU memory used for processing `token_count` tokens.
    """
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

    dummy_token_id = tokenizer.encode("the")[0]
    input_ids = torch.tensor([[dummy_token_id] * token_count], device="cuda")

    with torch.no_grad():
        _ = model(input_ids)

    return torch.cuda.max_memory_allocated() / 1024**2

# Set list of tokens to process
token_counts = [2**i for i in range(0, 14)]
baseline = measure_memory(1)

# Compute overheads
overheads = []
for n in token_counts:
    mem = measure_memory(n)
    overheads.append(mem - baseline)
    print(f"{n} tokens → overhead: {mem - baseline:.2f} MB")

# Plot graph
plt.plot(token_counts, overheads, marker="o")
plt.title("Memory Overhead vs Token Count")
plt.xlabel("Number of Tokens")
plt.ylabel("Overhead Memory (MB)")
plt.grid(True)
plt.show()

## **3. Comment transformer un LLM en un assistant ?**

Un LLM n'est qu'un moteur à générer du texte. Ce n'est pas encore un assistant qui répond à nos questions ! Dans l'exemple suivant, observez comme le LLM continue notre séquence de tokens par un texte probable (un message sur un blog) sans répondre à la question.

In [ ]:
# Define prompt
prompt = "How to remove an item from a list in python?"

# Generate text continuation
prediction = generator(
    prompt,
    do_sample=False,
    temperature=None,
    top_p=None,
    max_new_tokens=30
)

Pour transformer un LLM en assistant, il faut utiliser deux astuces qui vont permettre de mieux contrôler le texte généré.

### 3.1 Chat prompt template

Nous voulons intéragir avec le LLM sous la forme d'une conversation dans laquelle il répond aux questions que l'on pose. L'astuce consiste à modifier la structure de notre prompt pour faire comprendre au LLM qu'il se trouve dans une conversation et qu'il joue le rôle de l'assistant.

In [ ]:
prompt = """
USER:
    In python, how to compute the number of items in a list?
ASSISTANT:
    You can use the following function: `len(my_list)`.
USER:
    How to remove an item from a list in python?
ASSISTANT:
"""

# Generate text continuation
output = generator(
    prompt,
    do_sample=False,
    temperature=None,
    top_p=None,
    max_new_tokens=40
)

Le LLM comprend qu'après le token `ASSISTANT` se trouve une réponse à la question posée après le token `USER`.

### 3.2 Arrêter la génération à la fin de la réponse

Pour que le LLM s'arrête à la fin de la réponse `ASSISTANT`, on peut ajouter un token `FIN` pour signifier la fin d'un message puis arrêter la génération lorsque ce token est généré.

In [ ]:
prompt = """
USER:
    In python, how to compute the number of items in a list?
FIN
ASSISTANT:
    You can use the following function: `len(my_list)`.
FIN
USER:
    How to remove an item from a list in python?
FIN
ASSISTANT:
"""

# Generate text continuation
output = generator(
    prompt,
    do_sample=False,
    temperature=None,
    top_p=None,
    max_new_tokens=30,
    tokenizer=tokenizer,
    stop_strings="FIN" # stop generation when this token is sampled
)

---

**Question**

Selon vous, comment est gérée la mémoire de la conversation ?

Qu'est-ce que cela implique en terme de temps de calcul / coûts ?

---